In [1]:
# Import all libraries
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings

warnings.filterwarnings('ignore')
np.set_printoptions(precision=4) #tells you till 4 decimal value a float number can be

In [2]:
#Read the data using pandas
data = pd.read_csv('./data/UpdatedResumeDataSet.csv')
data.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [3]:
data['Category'].value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Data Science                 40
Mechanical Engineer          40
Sales                        40
ETL Developer                40
Blockchain                   40
Operations Manager           40
Arts                         36
Database                     33
PMO                          30
Health and fitness           30
Electrical Engineering       30
DotNet Developer             28
Business Analyst             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [4]:
data.describe()

,Category,Resume
count,962,962
unique,25,166
top,Java Developer,"Technical Skills Web Technologies: Angular JS,..."
freq,84,18


In [5]:
data = data[data.Category != 'Testing']

In [6]:
data_size= len(data)

In [7]:
data['Resume'][0]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [8]:
# As u can see in the above data , 
#lots of cleaning is required so lets go for cleanig the data

In [9]:
stopwords = set(stopwords.words('english')+['``',"''"])
stopwords

{"''",
 '``',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over

In [10]:
def clear_text(txt):
    clean_text = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    clean_text = re.sub('RT|cc', ' ', clean_text)  # remove RT and cc
    clean_text = re.sub('#\S+', '', clean_text)  # remove hashtags
    clean_text = re.sub('@\S+', '  ', clean_text)  # remove mentions
    clean_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', clean_text)  # remove punctuations
    clean_text = re.sub(r'[^\x00-\x7f]',r' ', clean_text) 
    clean_text = re.sub('\s+', ' ', clean_text)  # remove extra whitespace
    clean_text = clean_text.lower()  # convert to lowercase
    clean_text_tokens = word_tokenize(clean_text)  # tokenize
    filtered_text = [w for w in clean_text_tokens if not w in stopwords]  # remove stopwords
    return ' '.join(filtered_text)

In [11]:
#applying the above function to data
data["cleaned_resume"]= data.Resume.apply(lambda x:clear_text(x))

In [12]:
print(data['cleaned_resume'][0])

skills programming languages python pandas numpy scipy scikit learn matplotlib sql java javascript jquery machine learning regression svm na bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural nets database visualizations mysql sqlserver cassandra hbase elasticsearch d3 js dc js plotly kibana matplotlib ggplot tableau others regular expression html css angular 6 logstash kafka python flask git docker computer vision open cv understanding deep learning education details data science assurance associate data science assurance associate ernst young llp skill details javascript exprience 24 months jquery exprience 24 months python exprience 24 monthscompany details company ernst young llp description fraud investigations dispute services assurance technology assisted review tar technology assisted review assists elerating review process run analy

In [13]:
# separating the label and features
features = data["cleaned_resume"].values
labels_originals = data["Category"].values
labels = labels_originals[:]

In [14]:
#making labels not spaceSeparated
for i in range(data_size):
    labels[i]= str(labels[i].lower())
    labels[i]= labels[i].replace(" ","")
    

In [15]:
features , labels = shuffle(features, labels)
print(features[0])
print(labels[0])

core competencies maintain processes ensure project management documentation reports plans relevant urate complete report automation dashboard preparation sharing feedbacks basis performance project manager forecasting data regarding future risks project changes updating delivery team timely basis good understanding project management lifecycle proven excellence risk management control good understanding software development lifecycle sdlc ability synthesize qualitative quantitative data quickly draw meaningful insights knowledge programme project management methodologies full project reporting governance ability work different cross functional stakeholders establish ensure reliable productive working relationship strong time management organizational skills multitasking skills ability meet deadlines computer skills ce ification advance knowledge ms office 2013 macros skills strategic thinking decision making ability sound analytical skills multi tasking skills fast paced environment l

In [16]:
#Splitting the data into train and test data
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels,test_size=0.2, random_state =42)

In [17]:
print(train_features.shape, train_labels.shape, test_features.shape, test_labels.shape)

(713,) (713,) (179,) (179,)


In [18]:
train_features[0]

'technicalskills springmvc hibernate jdbc java j2ee azureweb suntechnologies services jsp struts servlet restapi scripting javascript ajax html json opensourcetechnologies php database mssql mysql oracle webservers apachetomcatserver internettechnologiesand onesignalwebpushnotifications azurewebservices frameworks ionic html json operatingsystem windowsserver2012r2 winxp 7 8 1 10 linux mac os education details august 2013 july 2016 computer engineering nashik maharashtra late g n sapkal coe nashik july 2009 june 2013 diploma computer technology nashik maharashtra k k wagh polytechnic nashik java developer java developer skill details ajax exprience 12 months database exprience 24 months html exprience 24 months j2ee exprience 6 months java exprience 24 months spring mvc exprience 12 months ionic 3 exprience 6 months angular js exprience 6 months spring exprience less 1 year months java exprience less 1 year monthscompany details company replete business solutions pvt ltd description wo

In [19]:
test_features[0]

'skills 1 ms office 2 good communication skills convincing power 3 knowledge sales marketing 4 customer sales management 5 talent management 6 direct sales management strengths 1 dedication dependability 2 flexibility able work taxing demanding conditions reasonably stretchable time slots 3 team work able work cooperatively team lead team su ess education details sales manager skill details marketing exprience 6 months direct sales exprience 36 months marketing exprience 6 months sales exprience 36 months sales exprience 36 monthscompany details company bajaj finance description company bajaj finance description sbi bank office assistence 1 year k g n shope office assistence 6 month vodafone telle caller 1 year bajaj finserv ltd assistant sales manager 11 month square capital sales manager 5 month present working bajaj housing finance ltd assistant sales manager'

In [20]:
#tokenization is very important part of nlp as machine dont understand words so it must get a number (here token is a number)
feature_tokenizer = Tokenizer(num_words=6000, oov_token='<OOV>')
feature_tokenizer.fit_on_texts(features)

feature_index = feature_tokenizer.word_index
print(dict(list(feature_index.items())))

{'<OOV>': 1, 'exprience': 2, 'company': 3, 'project': 4, 'months': 5, 'description': 6, 'details': 7, 'data': 8, '1': 9, 'management': 10, 'team': 11, 'year': 12, '6': 13, 'system': 14, 'maharashtra': 15, 'database': 16, 'development': 17, 'business': 18, 'less': 19, 'sql': 20, 'ltd': 21, 'using': 22, 'developer': 23, 'client': 24, 'java': 25, 'skill': 26, 'testing': 27, 'application': 28, 'pune': 29, 'engineering': 30, 'january': 31, 'services': 32, 'work': 33, 'skills': 34, 'c': 35, 'test': 36, 'software': 37, 'sales': 38, 'responsibilities': 39, 'pvt': 40, 'operations': 41, 'requirements': 42, 'reports': 43, 'education': 44, 'server': 45, 'technical': 46, 'process': 47, 'india': 48, 'customer': 49, 'various': 50, 'design': 51, 'technologies': 52, 'university': 53, 'working': 54, 'python': 55, 'monthscompany': 56, '2': 57, 'environment': 58, 'web': 59, 'like': 60, 'automation': 61, 'college': 62, 'support': 63, 'mysql': 64, 'worked': 65, 'role': 66, 'technology': 67, 'activities': 68

In [21]:
#lets try this on train and test datasets
train_features_sequences = feature_tokenizer.texts_to_sequences(train_features)
test_features_sequences = feature_tokenizer.texts_to_sequences(test_features)
print(train_features_sequences[0])
print("----------------------------------------------------------------------------")
print(test_features_sequences[0])

[3305, 3306, 405, 938, 25, 527, 3307, 3308, 32, 379, 1927, 585, 3309, 176, 128, 223, 122, 612, 3310, 397, 16, 1735, 64, 93, 3311, 3312, 3313, 3314, 3315, 2201, 1507, 122, 612, 3316, 3317, 3318, 169, 275, 9, 191, 133, 1359, 656, 44, 7, 633, 343, 583, 137, 86, 30, 552, 15, 1659, 441, 969, 3319, 1440, 552, 583, 609, 204, 343, 232, 86, 67, 552, 15, 477, 477, 2327, 749, 552, 25, 23, 25, 23, 26, 7, 223, 2, 262, 5, 16, 2, 166, 5, 122, 2, 166, 5, 527, 2, 13, 5, 25, 2, 166, 5, 321, 351, 2, 262, 5, 1507, 80, 2, 13, 5, 490, 401, 2, 13, 5, 321, 2, 19, 9, 12, 5, 25, 2, 19, 9, 12, 56, 7, 3, 3320, 18, 142, 40, 21, 6, 54, 25, 23, 321, 351, 64, 1735, 25, 527, 223, 128, 1507, 80, 228, 490, 401, 87, 52]
----------------------------------------------------------------------------
[34, 9, 111, 140, 57, 152, 171, 34, 4647, 160, 80, 76, 38, 238, 79, 49, 38, 10, 119, 1001, 10, 13, 637, 38, 10, 1040, 9, 2090, 4648, 57, 1621, 1062, 33, 4649, 4650, 1325, 4651, 4652, 69, 4653, 80, 11, 33, 1062, 33, 4654, 11, 218,

In [22]:
# now lets tokenize label dataset 
label_tokenizer = Tokenizer(lower=True)
label_tokenizer.fit_on_texts(labels)

label_index = label_tokenizer.word_index
print(dict(list(label_index.items())))
# Print example label encodings from train and test datasets
train_label_sequences = label_tokenizer.texts_to_sequences(train_labels)
print(train_label_sequences[0])

test_label_sequences = label_tokenizer.texts_to_sequences(test_labels)
print(test_label_sequences[0])

{'javadeveloper': 1, 'devopsengineer': 2, 'pythondeveloper': 3, 'webdesigning': 4, 'hr': 5, 'hadoop': 6, 'etldeveloper': 7, 'mechanicalengineer': 8, 'datascience': 9, 'blockchain': 10, 'sales': 11, 'operationsmanager': 12, 'arts': 13, 'database': 14, 'pmo': 15, 'electricalengineering': 16, 'healthandfitness': 17, 'businessanalyst': 18, 'dotnetdeveloper': 19, 'automationtesting': 20, 'networksecurityengineer': 21, 'civilengineer': 22, 'sapdeveloper': 23, 'advocate': 24}
[1]
[11]


In [23]:
# lets make the lenght of each sequence of same length using pad_sequence be keras
max_length = 300
trunc_type = 'post'
pad_type ='post'

train_feature_padded = pad_sequences(train_features_sequences, maxlen=max_length, padding=pad_type, truncating = trunc_type)
test_feature_padded = pad_sequences(test_features_sequences, maxlen=max_length, padding=pad_type, truncating = trunc_type)

#print example padded 
print(train_feature_padded[0])
print("_________________________________")
print(test_feature_padded[0])

[3305 3306  405  938   25  527 3307 3308   32  379 1927  585 3309  176
  128  223  122  612 3310  397   16 1735   64   93 3311 3312 3313 3314
 3315 2201 1507  122  612 3316 3317 3318  169  275    9  191  133 1359
  656   44    7  633  343  583  137   86   30  552   15 1659  441  969
 3319 1440  552  583  609  204  343  232   86   67  552   15  477  477
 2327  749  552   25   23   25   23   26    7  223    2  262    5   16
    2  166    5  122    2  166    5  527    2   13    5   25    2  166
    5  321  351    2  262    5 1507   80    2   13    5  490  401    2
   13    5  321    2   19    9   12    5   25    2   19    9   12   56
    7    3 3320   18  142   40   21    6   54   25   23  321  351   64
 1735   25  527  223  128 1507   80  228  490  401   87   52    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [24]:
#making the model
vocab_size=6000
embedding_dim= 64


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= embedding_dim, input_length=1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 64)             384000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1625      
Total params: 459,929
Trainable params: 459,929
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

final_train= np.array(train_feature_padded)
final_test = np.array(test_feature_padded)


final_train_label = np.array(train_label_sequences)
final_test_label = np.array(test_label_sequences)

In [26]:
num_epochs= 12
history =model.fit(final_train, final_train_label,epochs= num_epochs, validation_data=(final_test, final_test_label), verbose=2)

Epoch 1/12
23/23 - 5s - loss: 3.1808 - accuracy: 0.1725 - val_loss: 3.0852 - val_accuracy: 0.1788
Epoch 2/12
23/23 - 4s - loss: 2.8884 - accuracy: 0.1964 - val_loss: 2.6274 - val_accuracy: 0.2291
Epoch 3/12
23/23 - 4s - loss: 2.5020 - accuracy: 0.3450 - val_loss: 2.2911 - val_accuracy: 0.3352
Epoch 4/12
23/23 - 4s - loss: 2.0040 - accuracy: 0.4614 - val_loss: 1.7901 - val_accuracy: 0.4916
Epoch 5/12
23/23 - 4s - loss: 1.6543 - accuracy: 0.5288 - val_loss: 1.4903 - val_accuracy: 0.5531
Epoch 6/12
23/23 - 4s - loss: 1.2249 - accuracy: 0.6522 - val_loss: 1.1367 - val_accuracy: 0.6536
Epoch 7/12
23/23 - 4s - loss: 0.8883 - accuracy: 0.7798 - val_loss: 0.8382 - val_accuracy: 0.8101
Epoch 8/12
23/23 - 4s - loss: 0.6176 - accuracy: 0.8513 - val_loss: 0.5631 - val_accuracy: 0.8883
Epoch 9/12
23/23 - 4s - loss: 0.4415 - accuracy: 0.9088 - val_loss: 0.4242 - val_accuracy: 0.9162
Epoch 10/12
23/23 - 4s - loss: 0.3118 - accuracy: 0.9523 - val_loss: 0.2929 - val_accuracy: 0.9665
Epoch 11/12
23/23 -

In [27]:
# print example feature and its correct label
print(test_features[5])
print(test_labels[5])

hobbies playing chess solving rubik cube watching series languages english hindi marathi education details january 2014 january 2017 bachelor degree information technology first class pune maharashtra jspm jayawantrao sawant college engineering january 2010 january 2014 diploma information technology first class nashik maharashtra k k wagh polytechnic january 2010 ssc first class nashik maharashtra new era english school blockchain developer blockchain developer corpcloud global services pvt ltd skill details blockchain exprience 6 months smart contracts exprience 6 months dapps exprience 6 months mean stack exprience 12 monthscompany details company corpcloud global services pvt ltd description worked productively team identify requirements proposed ideas enhancing product developing managing user blockchain ount wallets transactions regularly monitoring smooth executions blockchain transactions wallet functions along identifying correcting possible errors writing smart contracts api 

In [28]:
# Create padded sequence for example
resume_example = 'software proficiency languages basics c sql pl sql java javaee javascript html css jquery mysql spring hibernate software tools xillinx modelsim matlab multisim operating systems windows xp vista 07 08 ubuntu project profile b e project fpga implementation team size 4 role programmer aes algorithm aes advanced encryption standard used cryptography protect data encrypted secret key e project sorting robot team size 3 role mechanism designer tcs 230 sensor sorts rgb color balls ording color diploma project rfid based student team size 4 role interface attendance system using gsm student show rfid card message send via gsm parent ward present education details may 2016 b e savitribai phule pune maharashtra pune university march 2010 c maharashtra board devops engineer skill details c exprience 6 months c exprience 6 months sql exprience 6 months pl sql exprience 6 months core java exprience 6 months javascript exprience less 1 year months html exprience less 1 year months css exprience less 1 year months jquery exprience less 1 year months javaee exprience less 1 year months mysql exprience less 1 year months python exprience 6 monthscompany details company parkar consulting labs description working devops team parkar consulting labs hands aws well python'
example_sequence = feature_tokenizer.texts_to_sequences([resume_example])
example_padded = pad_sequences(example_sequence, maxlen=max_length, padding=pad_type, truncating=trunc_type)
example_padded = np.array(example_padded)
print(example_padded)

[[  37 1285  225 1969   35   20  805   20   25 1559  128  122  198  130
    64  321  405   37  108 2657 2658 1307 2659  163   90  102  494 1286
  1355 1524  927    4  425   92   70    4 2660  231   11  282   79   66
  2281 1560 1875 1560 1145 2661  355   71 2662 2427    8 1688 2663  141
    70    4 1963 1146   11  282   80   66 2111  439 1451 2664 2212 2665
  2666 1423 2667  684 1423  232    4 1561   72  519   11  282   79   66
   513 1453   14   22 1356  519 1398 1561  883 1399 1424  661 1356 1597
  2668  495   44    7  251  137   92   70  811  744   29   15   29   53
   961  294   35   15  213  263   85   26    7   35    2   13    5   35
     2   13    5   20    2   13    5  805   20    2   13    5  270   25
     2   13    5  128    2   19    9   12    5  122    2   19    9   12
     5  198    2   19    9   12    5  130    2   19    9   12    5 1559
     2   19    9   12    5   64    2   19    9   12    5   55    2   13
    56    7    3 1562  934 1563    6   54  263   11 1562  934 15

In [29]:
# Make a prediction
prediction = model.predict(example_padded)

In [30]:
# Verify that prediction has correct format
print(prediction[0])
print(len(prediction[0]))  # should be 25
print(np.sum(prediction[0]))  # should be 1

[4.7209e-09 1.7606e-02 9.3036e-01 1.9111e-07 2.8279e-05 2.4996e-03
 6.1256e-05 1.8930e-06 8.9855e-07 2.2438e-02 1.6310e-02 1.0956e-11
 1.0112e-12 1.2059e-08 1.6521e-03 1.9287e-09 4.8723e-05 2.4558e-09
 2.4309e-10 1.7581e-05 6.9037e-03 1.8313e-03 2.6311e-10 2.0957e-04
 2.7782e-05]
25
0.99999994


In [31]:
# Find maximum value in prediction and its index
print(max(prediction[0]))  # confidence in prediction (as a fraction of 1)
print(np.argmax(prediction[0])) 

0.9303625
2


In [32]:
#As here u can see we got the argmax as 2 and 2 is the number in labelSequene for devopsengineer

In [33]:
# Save feature tokenizer
with open('feature_tokenizer.pickle', 'wb') as handle:
    pickle.dump(feature_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
# Save reverse dictionary of labels to encodings
label_to_encoding = dict(list(label_index.items()))
print(label_to_encoding)

encoding_to_label = {}
for k, v in label_to_encoding.items():
  encoding_to_label[v] = k
print(encoding_to_label)

with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(encoding_to_label, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'javadeveloper': 1, 'devopsengineer': 2, 'pythondeveloper': 3, 'webdesigning': 4, 'hr': 5, 'hadoop': 6, 'etldeveloper': 7, 'mechanicalengineer': 8, 'datascience': 9, 'blockchain': 10, 'sales': 11, 'operationsmanager': 12, 'arts': 13, 'database': 14, 'pmo': 15, 'electricalengineering': 16, 'healthandfitness': 17, 'businessanalyst': 18, 'dotnetdeveloper': 19, 'automationtesting': 20, 'networksecurityengineer': 21, 'civilengineer': 22, 'sapdeveloper': 23, 'advocate': 24}
{1: 'javadeveloper', 2: 'devopsengineer', 3: 'pythondeveloper', 4: 'webdesigning', 5: 'hr', 6: 'hadoop', 7: 'etldeveloper', 8: 'mechanicalengineer', 9: 'datascience', 10: 'blockchain', 11: 'sales', 12: 'operationsmanager', 13: 'arts', 14: 'database', 15: 'pmo', 16: 'electricalengineering', 17: 'healthandfitness', 18: 'businessanalyst', 19: 'dotnetdeveloper', 20: 'automationtesting', 21: 'networksecurityengineer', 22: 'civilengineer', 23: 'sapdeveloper', 24: 'advocate'}


In [35]:
print(encoding_to_label[np.argmax(prediction[0])])

devopsengineer


In [36]:
import PyPDF2

In [37]:
def readFromResume(filepath):
    # creating a pdf file object 
    pdfFileObj = open(filepath, 'rb') 

    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    # creating a page object 
    pageObj = pdfReader.getPage(0) 

    # extracting text from page 
    extractedTexts = pageObj.extractText()
    
    # closing the pdf file object 
    pdfFileObj.close() 
    return extractedTexts

In [38]:
texts=readFromResume('resume.pdf')

In [39]:
print(texts)

NITIN SUNIL CHAUBE
 
Course : BE Computers
 
Contact: 7021027955 
Gmail:
nitinchaube1@gmail.com
 
Github
:
https://github.com/nitinchaube
 
 
ACADEMIC DETAILS
 
COURSE
 
SPECIALIZATION
 
INSTITUTE/COLLEGE
 
BOARD/UNIVERSITY
 
% / CGPA
 
YEAR
 
BE
 
Computer Engineering
 
FR. Conceicao Rodrigues 
College
 
University of Mumbai
 
8.08(till 5th sem)
 
2022
 
XII
 
Science
 
Thomas 
Baptista junior college
 
Maharashtra State 
Board
 
85.85
 
2018
 
X
 
General
 
St. Francis High School
 
Maharashtra State 
Board
 
88.80
 
2016
 
 
SKILLS HIGHLIGHT
 
Languages :                      
C++, Java, Mongodb, Python
 
Web Technologies :         
HTML, Javascripts, CSS, React, Nodejs , Github , 
 
DataScienceLibraries :    
Postman Numpy, Pandas, Sklearn
 
 
PROJECTS
 
 

 

 
 
-
(
 
Link
 
)                                                                                                    
                                                    
 
May2020 
-
 
Aug2020
                              

In [40]:
def make_predictions(resume_example):
    example_sequence = feature_tokenizer.texts_to_sequences([resume_example])
    example_padded = pad_sequences(example_sequence, maxlen=max_length, padding=pad_type, truncating=trunc_type)
    example_padded = np.array(example_padded)
    # Make a prediction
    prediction = model.predict(example_padded)
    print("THE JOB WHICH FIT THIS RESUME IS :   ",encoding_to_label[np.argmax(prediction[0])] )

In [41]:
make_predictions(texts)

THE JOB WHICH FIT THIS RESUME IS :    devopsengineer


In [42]:
justinsText=readFromResume('JustinsResume.pdf')

In [43]:
make_predictions(justinsText)

THE JOB WHICH FIT THIS RESUME IS :    advocate
